# 🚀 API Phân tích Cảm xúc (HTML + LLM + Flask) 🚀

Notebook này sẽ khởi tạo một **ứng dụng web** hoàn chỉnh:
1.  **Frontend HTML:** Một trang web để nhập review.
2.  **Backend Flask:** Xử lý yêu cầu.
3.  **ABSA Model:** Chạy model ViT5-large của bạn để phân tích.
4.  **LLM Summarizer:** Dùng một model ViT5-base để tóm tắt kết quả thành ngôn ngữ tự nhiên.
5.  **Ngrok:** Đưa trang web ra Internet.

## 🛑 VIỆC CẦN LÀM (Rất quan trọng!)

Trước khi chạy, bạn **BẮT BUỘC** phải làm 3 việc sau:

### 1. Upload Model Checkpoints (.ckpt)

1.  Tạo một **Kaggle Dataset** MỚI.
2.  Upload 2 file model đã huấn luyện của bạn (ví dụ: `restaurant_best_model.ckpt` và `hotel_best_model.ckpt`) lên dataset đó.
3.  Thêm dataset này vào notebook Kaggle bằng cách nhấn vào **"Add Data"** (ở thanh bên phải).

### 2. Lấy Ngrok Auth Token

1.  Đăng nhập vào [Ngrok Dashboard](https://dashboard.ngrok.com/login).
2.  Ở mục **"Your Authtoken"**, copy cái token của bạn.
3.  Dán token này vào **Cell 2** ở biến `NGROK_AUTH_TOKEN`.

### 3. Cập nhật Cấu hình Model

Sau khi thêm Kaggle Dataset (ở Bước 1), bạn sẽ thấy đường dẫn của nó (ví dụ: `/kaggle/input/my-models`). 

Hãy cập nhật **Cell 2** ở biến `MODEL_CONFIGS`:

* **`ckpt_path`**: Sửa lại đường dẫn cho đúng với file model của bạn (ví dụ: `/kaggle/input/my-models/restaurant_best_model.ckpt`).
* **`aspect_columns`**: Bạn phải **dán ĐÚNG và ĐỦ** danh sách các cột aspect của bạn vào đây. Tôi đã để các giá trị ví dụ.

In [1]:
# === Cell 1: Cài đặt thư viện ===
# Thêm 'sentencepiece' cho model LLM tóm tắt (ViT5)

print("Đang cài đặt các thư viện chính...")
!pip install flask pyngrok pytorch-lightning transformers pandas sentencepiece -q

print("--- SỬA LỖI: Cài đặt lại Protobuf (Nếu cần) ---")
!pip uninstall -y protobuf
!pip install protobuf==3.20.3 -q

print("Đã cài đặt xong.")
print("LƯU Ý: Vui lòng BỎ QUA các lỗi 'Unable to register' (nếu có).")

Đang cài đặt các thư viện chính...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account 

## 🛑 QUAN TRỌNG: KHỞI ĐỘNG LẠI KERNEL

Sau khi Cell 1 chạy xong, hãy **KHỞI ĐỘNG LẠI KERNEL** để áp dụng bản `protobuf` đã sửa.

Trên thanh menu, chọn **Run -> Restart Session**.

Sau đó, hãy **chạy lại Cell 1** một lần nữa (để load thư viện), rồi **chạy tiếp từ Cell 2**.

In [2]:
# === Cell 2: Cấu hình (SỬA LỖI ĐƯỜNG DẪN CKPT) ===
import os
import json

# 1. Dán NGROK Auth Token của bạn vào đây
NGROK_AUTH_TOKEN = "33xssoeiIZT6i4jrZb0ZAOECVz6_7garq9w8GQ8kri2Ni92J"  # <--- THAY THẾ!

# 2. Cấu hình Model Checkpoints và Aspects
#    SỬA LẠI 'ckpt_path' cho đúng với model của bạn
MODEL_CONFIGS = {
    "restaurant": {
        # !!! KIỂM TRA KỸ DÒNG NÀY !!!
        "ckpt_path": "/kaggle/input/absa-model/absa_restaurant_best_model.ckpt",  # <--- PHẢI LÀ FILE CỦA RESTAURANT (12 aspect)
        "aspect_columns": [
            'AMBIENCE#GENERAL', 'DRINKS#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE&OPTIONS', 
            'FOOD#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE&OPTIONS', 'LOCATION#GENERAL', 
            'RESTAURANT#GENERAL', 'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#PRICES', 
            'SERVICE#GENERAL'
        ]
    },
    "hotel": {
        # !!! KIỂM TRA KỸ DÒNG NÀY !!!
        "ckpt_path": "/kaggle/input/absa-model/absa_hotel_best_model.ckpt",  # <--- PHẢI LÀ FILE CỦA HOTEL (34 aspect)
        "aspect_columns": [
            'FACILITIES#CLEANLINESS', 'FACILITIES#COMFORT', 'FACILITIES#DESIGN&FEATURES', 
            'FACILITIES#GENERAL', 'FACILITIES#MISCELLANEOUS', 'FACILITIES#PRICES', 
            'FACILITIES#QUALITY', 'FOOD&DRINKS#MISCELLANEOUS', 'FOOD&DRINKS#PRICES', 
            'FOOD&DRINKS#QUALITY', 'FOOD&DRINKS#STYLE&OPTIONS', 'HOTEL#CLEANLINESS', 
            'HOTEL#COMFORT', 'HOTEL#DESIGN&FEATURES', 'HOTEL#GENERAL', 'HOTEL#MISCELLANEOUS', 
            'HOTEL#PRICES', 'HOTEL#QUALITY', 'LOCATION#GENERAL', 'ROOMS#CLEANLINESS', 'ROOMS#COMFORT',
            'ROOMS#DESIGN&FEATURES', 'ROOMS#GENERAL', 'ROOMS#MISCELLANEOUS', 'ROOMS#PRICES', 
            'ROOMS#QUALITY', 'ROOM_AMENITIES#CLEANLINESS', 'ROOM_AMENITIES#COMFORT', 'ROOM_AMENITIES#DESIGN&FEATURES', 
            'ROOM_AMENITIES#GENERAL', 'ROOM_AMENITIES#MISCELLANEOUS', 'ROOM_AMENITIES#PRICES', 
            'ROOM_AMENITIES#QUALITY', 'SERVICE#GENERAL'
        ]
    }
}

# 3. Cấu hình chung
# Đồng bộ tokenizer với encoder (cả 2 đều là 'large')
TOKENIZER_NAME = "VietAI/vit5-large"
LLM_SUMMARIZER_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

# Tắt cảnh báo của Tokenizers
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

print(f"Cấu hình đã được tải.")
print(f"SỬ DỤNG TOKENIZER: {TOKENIZER_NAME}") 
print(f"Restaurant aspects: {len(MODEL_CONFIGS['restaurant']['aspect_columns'])}")
print(f"Hotel aspects: {len(MODEL_CONFIGS['hotel']['aspect_columns'])}")

Cấu hình đã được tải.
SỬ DỤNG TOKENIZER: VietAI/vit5-large
Restaurant aspects: 12
Hotel aspects: 34


In [3]:
# === Cell 3: Ghi file Định nghĩa Model (model_definition.py) ===
# Cell này không thay đổi, chứa kiến trúc model của bạn.

In [4]:
!mkdir -p templates

In [5]:
%%writefile model_definition.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from transformers import T5EncoderModel
import math

# --- Class Loss (Cần để tải model) ---
class AsymmetricLossOptimized(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=False):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps
        self.targets = self.anti_targets = self.xs_pos = self.xs_neg = self.asymmetric_w = self.loss = None

    def forward(self, x, y):
        self.targets = y
        self.anti_targets = 1 - y
        self.xs_pos = torch.sigmoid(x)
        self.xs_neg = 1.0 - self.xs_pos
        if self.clip is not None and self.clip > 0:
            self.xs_neg.add_(self.clip).clamp_(max=1)
        self.loss = self.targets * torch.log(self.xs_pos.clamp(min=self.eps))
        self.loss.add_(self.anti_targets * torch.log(self.xs_neg.clamp(min=self.eps)))
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            self.xs_pos = self.xs_pos * self.targets
            self.xs_neg = self.xs_neg * self.anti_targets
            self.asymmetric_w = torch.pow(1 - self.xs_pos - self.xs_neg,
                                          self.gamma_pos * self.targets + self.gamma_neg * self.anti_targets)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            self.loss *= self.asymmetric_w
        return -self.loss.sum() / x.size(0)

# --- Các Sub-module của Kiến trúc Model ---

class NeighborhoodAttention(nn.Module):
    def __init__(self, hidden_size: int, num_heads: int = 8, window_size: int = 7):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.window_size = window_size
        self.head_dim = hidden_size // num_heads
        self.qkv = nn.Linear(hidden_size, hidden_size * 3, bias=False)
        self.proj = nn.Linear(hidden_size, hidden_size)
        
    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        B, L, C = x.shape
        qkv = self.qkv(x).reshape(B, L, 3, self.num_heads, self.head_dim)
        q, k, v = qkv.permute(2, 0, 3, 1, 4)
        attn_output = []
        for i in range(L):
            start = max(0, i - self.window_size // 2)
            end = min(L, i + self.window_size // 2 + 1)
            q_i = q[:, :, i:i+1, :]
            k_window = k[:, :, start:end, :]
            v_window = v[:, :, start:end, :]
            scores = torch.matmul(q_i, k_window.transpose(-2, -1)) / math.sqrt(self.head_dim)
            attn_weights = F.softmax(scores, dim=-1)
            out_i = torch.matmul(attn_weights, v_window)
            attn_output.append(out_i)
        attn_output = torch.cat(attn_output, dim=2)
        attn_output = attn_output.transpose(1, 2).reshape(B, L, C)
        return self.proj(attn_output)

class StateConvBlock(nn.Module):
    def __init__(self, hidden_size: int):
        super().__init__()
        inner_dim = hidden_size * 2
        self.in_proj = nn.Linear(hidden_size, inner_dim * 2, bias=False)
        self.conv1d = nn.Conv1d(inner_dim, inner_dim, kernel_size=3, padding=1, groups=inner_dim)
        self.out_proj = nn.Linear(inner_dim, hidden_size, bias=False)
        self.norm = nn.LayerNorm(hidden_size)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_and_res = self.in_proj(x)
        x_inner, res = x_and_res.split(x_and_res.shape[-1] // 2, dim=-1)
        x_inner = self.conv1d(x_inner.transpose(1, 2)).transpose(1, 2)
        x_inner = F.silu(x_inner)
        y = x_inner * F.silu(res)
        output = self.out_proj(y)
        return x + self.norm(output)

class PositionBiasedAttention(nn.Module):
    def __init__(self, hidden_size: int, num_aspects: int):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_aspects = num_aspects
        self.attn = nn.MultiheadAttention(hidden_size, num_heads=8, batch_first=True)
        self.aspect_queries = nn.Parameter(torch.randn(num_aspects, hidden_size))
    def _create_position_bias(self, seq_len: int, device: torch.device) -> torch.Tensor:
        bias = torch.zeros(seq_len, seq_len, device=device)
        for i in range(seq_len):
            for j in range(seq_len):
                distance = abs(i - j)
                if distance > 5:
                    bias[i, j] = -10.0
        return bias
    def forward(self, x: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        B, L, D = x.shape
        pos_bias = self._create_position_bias(L, x.device)
        attn_out, _ = self.attn(x, x, x, attn_mask=pos_bias)
        aspect_features = []
        for b in range(B):
            seq_len = int(attention_mask[b].sum().item())
            valid_keys = attn_out[b, :seq_len].unsqueeze(0)
            queries = self.aspect_queries.unsqueeze(0)
            attn_score = torch.matmul(queries, valid_keys.transpose(-2, -1)) / math.sqrt(D)
            attn_weight = F.softmax(attn_score, dim=-1)
            pooled = torch.matmul(attn_weight, valid_keys)
            aspect_features.append(pooled.squeeze(0))
        return torch.stack(aspect_features)

class AdvancedCNN(nn.Module):
    def __init__(self, hidden_size: int, num_filters: int = 128):
        super().__init__()
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(hidden_size, num_filters, kernel_size=3, dilation=1, padding=1),
            nn.Conv1d(hidden_size, num_filters, kernel_size=3, dilation=2, padding=2),
            nn.Conv1d(hidden_size, num_filters, kernel_size=3, dilation=4, padding=4),
        ])
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fusion = nn.Linear(num_filters * len(self.conv_layers), hidden_size)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.transpose(1, 2)
        conv_outputs = []
        for conv in self.conv_layers:
            conv_out = F.relu(conv(x))
            pooled = self.pool(conv_out).squeeze(-1)
            conv_outputs.append(pooled)
        fused = torch.cat(conv_outputs, dim=-1)
        return self.fusion(fused)

# --- Class Model Chính ---
class AdvancedABSAModel(pl.LightningModule):
    def __init__(self, 
                 num_aspects=12, 
                 learning_rate=2e-5, 
                 encoder_name='VietAI/vit5-large',
                 use_neighborhood_attn=True,
                 use_state_conv=True,
                 use_position_attn=True,
                 use_advanced_cnn=True):
        super().__init__()
        self.save_hyperparameters() 
        
        self.num_aspects = num_aspects
        self.encoder = T5EncoderModel.from_pretrained(encoder_name)
        hidden_size = self.encoder.config.d_model

        self.use_neighborhood_attn = use_neighborhood_attn
        self.use_state_conv = use_state_conv
        self.use_position_attn = use_position_attn
        self.use_advanced_cnn = use_advanced_cnn

        if self.use_neighborhood_attn:
            self.neighborhood_attn = NeighborhoodAttention(hidden_size, window_size=7)
        if self.use_state_conv:
            self.state_conv_blocks = nn.ModuleList([StateConvBlock(hidden_size) for _ in range(2)]) # Mặc định là 2 khối như trong notebook
        if self.use_position_attn:
            self.position_biased_attn = PositionBiasedAttention(hidden_size, num_aspects)
        if self.use_advanced_cnn:
            self.advanced_cnn = AdvancedCNN(hidden_size)
        
        if self.use_position_attn and self.use_advanced_cnn:
             self.fusion = nn.Linear(hidden_size * 2, hidden_size)
        
        self.classifiers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_size, hidden_size // 2),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(hidden_size // 2, 4)
            ) for _ in range(num_aspects)
        ])
        
        self.criterion = AsymmetricLossOptimized()

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        encoder_output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        x = encoder_output.last_hidden_state
    
        if self.use_neighborhood_attn:
            x_block = self.neighborhood_attn(x, attention_mask)
        else:
            x_block = x
    
        if self.use_state_conv:
            for block in self.state_conv_blocks:
                x_block = block(x_block)
    
        if self.use_position_attn:
            aspect_features = self.position_biased_attn(x_block, attention_mask)
        else:
            aspect_features = x_block.mean(dim=1, keepdim=True).expand(-1, self.num_aspects, -1)
    
        if self.use_advanced_cnn:
            cnn_features = self.advanced_cnn(x_block)
        else:
            cnn_features = x_block.mean(dim=1)
    
        cnn_features = cnn_features.unsqueeze(1).expand(-1, self.num_aspects, -1)
        
        if self.use_position_attn and self.use_advanced_cnn:
            fused_features = torch.cat([aspect_features, cnn_features], dim=-1)
            fused_features = self.fusion(fused_features)
        elif self.use_position_attn:
            fused_features = aspect_features
        elif self.use_advanced_cnn:
            fused_features = cnn_features
        else:
            fused_features = x_block.mean(dim=1, keepdim=True).expand(-1, self.num_aspects, -1)
            
        logits = []
        for i, classifier in enumerate(self.classifiers):
            aspect_logits = classifier(fused_features[:, i, :])
            logits.append(aspect_logits)
        logits = torch.stack(logits, dim=1)
        return logits


Writing model_definition.py


In [6]:
# === Cell 4: Ghi file Logic Xử lý (app_logic.py) ===
# NÂNG CẤP: Thêm logic tải model LLM tóm tắt

In [7]:
%%writefile app_logic.py
import torch
from transformers import AutoTokenizer, pipeline
from model_definition import AdvancedABSAModel
import time

# Biến global để cache model (giữ model đã tải trong RAM)
loaded_models_cache = {}
loaded_summarizer_cache = None

IDX_TO_SENTIMENT = {0: 'None', 1: 'positive', 2: 'negative', 3: 'neutral'}
MAX_LENGTH = 128 # Giống như khi train

def get_model_components(model_type: str, model_configs: dict, tokenizer_name: str, device: torch.device):
    """Tải model ABSA và tokenizer. Tái sử dụng nếu đã tải."""
    if model_type in loaded_models_cache:
        print(f"Using cached model for: {model_type}")
        return loaded_models_cache[model_type]
    
    print(f"Cache miss. Loading model for: {model_type}...")
    start_time = time.time()
    
    try:
        config = model_configs[model_type]
        ckpt_path = config["ckpt_path"]
        aspect_columns = config["aspect_columns"]
    except KeyError:
        raise ValueError(f"Model type '{model_type}' không có trong cấu hình.")

    # 1. Tải Tokenizer (cache chung)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    
    # 2. Tải Model ABSA từ Checkpoint
    num_aspects = len(aspect_columns)
    try:
        model = AdvancedABSAModel.load_from_checkpoint(
            checkpoint_path=ckpt_path,
            map_location=device,
            num_aspects=num_aspects
        )
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy file checkpoint tại: {ckpt_path}")
        raise
    except Exception as e:
        print(f"LỖI khi tải checkpoint: {e}")
        raise
        
    model.eval()
    model.to(device)
    
    components = (model, tokenizer, aspect_columns)
    loaded_models_cache[model_type] = components
    
    end_time = time.time()
    print(f"Model for '{model_type}' loaded in {end_time - start_time:.2f} seconds.")
    return components

def run_prediction(review_text: str, model: AdvancedABSAModel, tokenizer, aspect_columns: list, device: torch.device) -> dict:
    """Chạy dự đoán ABSA và trả về dict kết quả."""
    
    encoding = tokenizer(
        review_text,
        truncation=True,
        padding='max_length',
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        
    preds = torch.argmax(logits, dim=-1).squeeze(0)
    preds_cpu = preds.cpu().numpy()
    
    results = {}
    for i, aspect_name in enumerate(aspect_columns):
        sentiment_idx = preds_cpu[i]
        sentiment_name = IDX_TO_SENTIMENT[sentiment_idx]
        
        if sentiment_name != 'None':
            friendly_name = aspect_name.replace('#', ' (').replace('_', ' ') + ')'
            if 'GENERAL' in friendly_name:
                friendly_name = friendly_name.replace(' (GENERAL)', '')
            results[friendly_name] = sentiment_name
            
    return results

# *** HÀM ĐÃ SỬA LỖI (1) ***
def get_summarizer(model_name: str, device: torch.device):
    """Tải model LLM (dưới dạng text-generation)."""
    global loaded_summarizer_cache
    if loaded_summarizer_cache:
        print("Using cached generator (LLM).")
        return loaded_summarizer_cache
    
    print(f"Loading TEXT-GENERATION pipeline for: {model_name}...")
    start_time = time.time()
    
    # SỬA LỖI: Dùng 'text-generation' cho model Qwen/Llama
    summarizer = pipeline(
        "text-generation", # <--- THAY ĐỔI QUAN TRỌNG
        model=model_name,
        device=0 if "cuda" in device.type else -1,
        # Thêm trust_remote_code cho các model Qwen (bắt buộc)
        trust_remote_code=True 
    )
    
    loaded_summarizer_cache = summarizer
    end_time = time.time()
    print(f"Text-generation pipeline loaded in {end_time - start_time:.2f} seconds.")
    return summarizer

# *** HÀM ĐÃ SỬA LỖI (2) ***
def generate_natural_response(structured_results: dict, original_review: str, summarizer) -> str:
    """Tạo câu trả lời tự nhiên bằng Qwen Chat Template."""
    if not structured_results:
        return "Cảm ơn bạn đã chia sẻ, nhưng tôi không tìm thấy khía cạnh nào cụ thể trong review này."

    # 1. Chuyển dict kết quả thành chuỗi
    analysis_parts = []
    for aspect, sentiment in structured_results.items():
        if sentiment == 'positive': sentiment_vn = 'tích cực'
        elif sentiment == 'negative': sentiment_vn = 'tiêu cực'
        else: sentiment_vn = 'trung tính'
        analysis_parts.append(f"{aspect}: {sentiment_vn}")
    analysis_str = ", ".join(analysis_parts)
    
    # 2. *** PROMPT MỚI (Dạy model theo khuôn mẫu của bạn) ***
    # Đây là "few-shot prompting" để ép model tuân thủ
    system_prompt = (
        "Bạn là một trợ lý AI, chỉ trả lời bằng tiếng Việt. "
        "Nhiệm vụ của bạn là chuyển danh sách phân tích (Input) thành một câu tóm tắt (Output) *duy nhất* theo khuôn mẫu: 'Người dùng cảm thấy [cảm xúc] về [khía cạnh]'. "
        "Khi có nhiều khía cạnh, hãy dùng từ nối (như 'và', 'nhưng', 'trong khi') để tạo thành một câu hoàn chỉnh.\n\n"
        "Ví dụ 1:\n"
        "Input: \"FOOD (QUALITY): tích cực\"\n"
        "Output: Người dùng cảm thấy tích cực về chất lượng đồ ăn.\n\n"
        "Ví dụ 2:\n"
        "Input: \"FOOD (QUALITY): tích cực, SERVICE (GENERAL): tiêu cực\"\n"
        "Output: Người dùng cảm thấy tích cực về chất lượng đồ ăn, nhưng tiêu cực về dịch vụ.\n\n"
        "Ví dụ 3:\n"
        "Input: \"SERVICE (GENERAL): tích cực, DRINKS (QUALITY): tích cực, FOOD (PRICES): trung tính\"\n"
        "Output: Người dùng cảm thấy tích cực về dịch vụ và chất lượng đồ uống, trong khi giá đồ ăn ở mức trung bình."
    )

    prompt = (
        f"<|im_start|>system\n{system_prompt}<|im_end|>\n"
        "<|im_start|>user\n"
        f"Input: \"{analysis_str}\"<|im_end|>\n"
        "<|im_start|>assistant\nOutput: " # Thêm "Output: " để kích hoạt (trigger)
    )
    
    # 3. Chạy LLM
    try:
        outputs = summarizer(
            prompt,
            max_new_tokens=100, # Giới hạn 100 token
            min_length=5,
            do_sample=False,
            num_beams=4,
            # Yêu cầu pipeline dừng khi gặp token kết thúc câu
            eos_token_id=summarizer.tokenizer.eos_token_id, 
            # Chỉ trả về phần text mới, không lặp lại prompt
            return_full_text=False 
        )
        # Lấy kết quả, xóa "Output: " (nếu lỡ có) và xóa khoảng trắng
        generated_text = outputs[0]['generated_text'].strip()
        if generated_text.startswith("Output:"):
            generated_text = generated_text[len("Output:"):].strip()
        return generated_text
        
    except Exception as e:
        print(f"Lỗi khi tóm tắt: {e}")
        return "Xin lỗi, tôi gặp lỗi khi tạo tóm tắt."

Writing app_logic.py


In [8]:
# === Cell 5: Ghi file HTML (templates/index.html) ===
# NÂNG CẤP: Tạo thư mục 'templates' và file HTML cho frontend

In [9]:
!mkdir -p templates

In [10]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="vi">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Bot Phân Tích Review</title>
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif; 
               background-color: #f4f7f6; margin: 0; padding: 20px; display: flex; justify-content: center; }
        .container { max-width: 700px; width: 100%; background-color: #ffffff; 
                     border-radius: 12px; box-shadow: 0 4px 12px rgba(0,0,0,0.05); padding: 30px; }
        h1 { color: #333; text-align: center; margin-top: 0; }
        form { display: flex; flex-direction: column; gap: 15px; }
        textarea, select { width: 100%; padding: 12px; border-radius: 8px; border: 1px solid #ddd; 
                           font-size: 16px; box-sizing: border-box; }
        button { background-color: #007aff; color: white; padding: 15px; border: none; 
                 border-radius: 8px; font-size: 16px; font-weight: 600; cursor: pointer; transition: background-color 0.2s; }
        button:hover { background-color: #005bb5; }
        button:disabled { background-color: #cddc39; cursor: not-allowed; }
        .loader { text-align: center; font-weight: 600; color: #555; display: none; margin-top: 20px; }
        .results { margin-top: 30px; border-top: 1px solid #eee; padding-top: 20px; }
        .results h2 { margin-top: 0; color: #007aff; }
        .results pre { background-color: #f0f0f0; padding: 15px; border-radius: 8px; 
                       white-space: pre-wrap; word-wrap: break-word; font-family: monospace; }
        .summary { background-color: #e6f7ff; border: 1px solid #b3e0ff; padding: 15px; 
                   border-radius: 8px; font-size: 1.1em; color: #0056b3; line-height: 1.6; }
        .review-echo { border-left: 4px solid #ccc; padding-left: 10px; font-style: italic; color: #555; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🤖 Bot Phân Tích Review</h1>
        <form id="review-form" action="/predict" method="POST">
            <label for="review">Nhập review của bạn:</label>
            <textarea id="review" name="review" rows="6" required>{{ results.review if results }}</textarea>

            <label for="model_type">Chọn loại hình:</label>
            <select id="model_type" name="model_type" required>
                <option value="restaurant" {% if results and results.model_type == 'restaurant' %}selected{% endif %}>Nhà hàng</option>
                <option value="hotel" {% if results and results.model_type == 'hotel' %}selected{% endif %}>Khách sạn</option>
            </select>

            <button id="submit-btn" type="submit">Phân tích</button>
        </form>
        
        <div id="loader" class="loader">
            Đang phân tích... (Lần đầu có thể mất 1-2 phút để tải model) ⏳
        </div>

        {% if results %}
        <div class="results">
            <h2>Kết quả Phân tích</h2>

            <h3>Tóm tắt của Bot:</h3>
            <p class="summary">{{ results.natural_summary }}</p>

            <h3>Review của bạn:</h3>
            <p class="review-echo">{{ results.review }}</p>

            <h3>Chi tiết (Dạng JSON):</h3>
            <pre>{{ results.structured_analysis | tojson(indent=4) }}</pre>
        </div>
        {% endif %}
    </div>

    <script>
        // Hiển thị loader khi form được submit
        document.getElementById('review-form').addEventListener('submit', function() {
            document.getElementById('submit-btn').disabled = true;
            document.getElementById('submit-btn').innerText = 'Đang xử lý...';
            document.getElementById('loader').style.display = 'block';
        });
    </script>
</body>
</html>

Writing templates/index.html


In [11]:
# === Cell 6: Ghi file API Chính (main_app.py) ===
# NÂNG CẤP: 
# 1. Sửa lỗi hardcode config (đọc từ biến 'MODEL_CONFIGS' của notebook)
# 2. Thêm route '/' để phục vụ file HTML
# 3. Nâng cấp route '/predict' để nhận cả JSON (API) và Form (HTML)
# 4. Gọi model LLM tóm tắt và trả về kết quả

In [12]:
# Dùng cách này để tiêm (inject) biến MODEL_CONFIGS từ Cell 2 vào file .py
# Tránh bị lỗi hardcode như phiên bản trước
file_content = f"""
import torch
from flask import Flask, request, jsonify, render_template
import app_logic
import threading
import json

# --- CẤU HÌNH ĐƯỢC TIÊM TỪ NOTEBOOK ---
MODEL_CONFIGS_FROM_NOTEBOOK = {json.dumps(MODEL_CONFIGS)}
TOKENIZER_NAME_FROM_NOTEBOOK = "{TOKENIZER_NAME}"
LLM_NAME_FROM_NOTEBOOK = "{LLM_SUMMARIZER_NAME}"
# -----------------------------------

app = Flask(__name__, template_folder='templates')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Flask App running on device: {{DEVICE}}")

# Lock để tránh 2 request cùng lúc tải model
model_load_lock = threading.Lock()

@app.route('/')
def home():
    # Phục vụ file templates/index.html
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        is_api_call = request.is_json
        
        if is_api_call:
            data = request.json
            review = data.get('review')
            model_type = data.get('model_type') # "restaurant" or "hotel"
        else:
            # Lấy từ form HTML
            data = request.form
            review = data.get('review')
            model_type = data.get('model_type')

        if not review or not model_type:
            error = {{"error": "Missing 'review' or 'model_type'"}}
            return jsonify(error), 400 if is_api_call else render_template('index.html', results=error)

        if model_type not in MODEL_CONFIGS_FROM_NOTEBOOK:
            error = {{"error": f"Invalid model_type. Choose from: {{list(MODEL_CONFIGS_FROM_NOTEBOOK.keys())}}"}}
            return jsonify(error), 400 if is_api_call else render_template('index.html', results=error)
        
        # --- Tải Model --- 
        # Dùng lock để đảm bảo an toàn khi tải model (thread-safe)
        with model_load_lock:
            # 1. Tải model ABSA
            model, tokenizer, aspect_cols = app_logic.get_model_components(
                model_type, MODEL_CONFIGS_FROM_NOTEBOOK, TOKENIZER_NAME_FROM_NOTEBOOK, DEVICE
            )
            # 2. Tải model LLM Tóm tắt
            summarizer = app_logic.get_summarizer(LLM_NAME_FROM_NOTEBOOK, DEVICE)
        
        # --- Chạy Inference ---
        # 1. Chạy model ABSA
        structured_results = app_logic.run_prediction(review, model, tokenizer, aspect_cols, DEVICE)
        
        # 2. Chạy model LLM Tóm tắt
        natural_summary = app_logic.generate_natural_response(structured_results, review, summarizer)
        
        # --- Trả kết quả ---
        final_results = {{
            "review": review,
            "model_type": model_type,
            "natural_summary": natural_summary,
            "structured_analysis": structured_results
        }}
        
        if is_api_call:
            # Nếu là API, trả JSON
            return jsonify(final_results)
        else:
            # Nếu là Form, render lại trang HTML với kết quả
            return render_template('index.html', results=final_results)

    except Exception as e:
        print(f"Error during prediction: {{e}}")
        error = {{"error": str(e)}}
        return jsonify(error), 500 if is_api_call else render_template('index.html', results=error)

def run_server():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
"""

# Ghi nội dung đã được chèn biến vào file
with open("main_app.py", "w", encoding="utf-8") as f:
    f.write(file_content)

print("Đã ghi file main_app.py và tiêm (inject) cấu hình thành công.")

Đã ghi file main_app.py và tiêm (inject) cấu hình thành công.


In [13]:
# === Cell 7: Ghi file API Chính (main_app.py) ===
# SỬA LỖI: Chỉ pre-load model Summarizer để tiết kiệm VRAM
# SỬA LỖI LẦN 3: Sửa IndentationError trong khối 'except'

# Dùng cách này để tiêm (inject) biến MODEL_CONFIGS từ Cell 2 vào file .py
file_content = f"""
import torch
from flask import Flask, request, jsonify, render_template
import app_logic
import threading
import json
import time

# --- CẤU HÌNH ĐƯỢC TIÊM TỪ NOTEBOOK ---
MODEL_CONFIGS_FROM_NOTEBOOK = {json.dumps(MODEL_CONFIGS)}
TOKENIZER_NAME_FROM_NOTEBOOK = "{TOKENIZER_NAME}"
LLM_NAME_FROM_NOTEBOOK = "{LLM_SUMMARIZER_NAME}"
# ----------------------------------

app = Flask(__name__, template_folder='templates')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Flask App running on device: {{DEVICE}}")

# --- KHỞI ĐỘNG VÀ TẢI TRƯỚC MODEL TÓM TẮT ---
# Chỉ tải LLM Summarizer (dùng chung) để tiết kiệm VRAM
def pre_load_summarizer():
    print("="*30)
    print("BẮT ĐẦU TẢI TRƯỚC MODEL SUMMARIZER (WARM-UP)...")
    
    start_llm = time.time()
    try:
        app_logic.get_summarizer(LLM_NAME_FROM_NOTEBOOK, DEVICE)
        # SỬA LỖI F-STRING (thêm {{...}})
        print(f"-> Đã tải xong LLM Summarizer (mất {{time.time() - start_llm:.2f}}s)")
    except Exception as e:
        print(f"!!! Lỗi khi tải LLM Summarizer: {{e}}")

    print("HOÀN TẤT TẢI TRƯỚC. Các model ABSA sẽ được tải khi có request đầu tiên.")
    print("Server Flask đang khởi động...")
    print("="*30)

pre_load_summarizer()
# ----------------------------------

# Lock để tránh 2 request cùng lúc tải model
model_load_lock = threading.Lock()

@app.route('/')
def home():
    # Phục vụ file templates/index.html
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        is_api_call = request.is_json
        
        if is_api_call:
            data = request.json
            review = data.get('review')
            model_type = data.get('model_type') # "restaurant" or "hotel"
        else:
            # Lấy từ form HTML
            data = request.form
            review = data.get('review')
            model_type = data.get('model_type')

        if not review or not model_type:
            error = {{"error": "Missing 'review' or 'model_type'"}}
            return jsonify(error), 400 if is_api_call else render_template('index.html', results=error)

        if model_type not in MODEL_CONFIGS_FROM_NOTEBOOK:
            error = {{"error": f"Invalid model_type. Choose from: {{list(MODEL_CONFIGS_FROM_NOTEBOOK.keys())}}"}}
            return jsonify(error), 400 if is_api_call else render_template('index.html', results=error)
        
        # --- Tải Model (Lazy-load) --- 
        # Dùng lock để đảm bảo an toàn khi tải model (thread-safe)
        with model_load_lock:
            # 1. Tải model ABSA (Sẽ tải nếu chưa có, hoặc dùng cache nếu đã tải)
            model, tokenizer, aspect_cols = app_logic.get_model_components(
                model_type, MODEL_CONFIGS_FROM_NOTEBOOK, TOKENIZER_NAME_FROM_NOTEBOOK, DEVICE
            )
            # 2. Lấy model LLM Tóm tắt (Đã được pre-load, chạy rất nhanh)
            summarizer = app_logic.get_summarizer(LLM_NAME_FROM_NOTEBOOK, DEVICE)
        
        # --- Chạy Inference ---
        structured_results = app_logic.run_prediction(review, model, tokenizer, aspect_cols, DEVICE)
        natural_summary = app_logic.generate_natural_response(structured_results, review, summarizer)
        
        # --- Trả kết quả ---
        final_results = {{
            "review": review,
            "model_type": model_type,
            "natural_summary": natural_summary,
            "structured_analysis": structured_results
        }}
        
        if is_api_call:
            return jsonify(final_results)
        else:
            return render_template('index.html', results=final_results)

    except Exception as e:
        # *** ĐÃ SỬA LỖI THỤT LỀ Ở 3 DÒNG DƯỚI (từ 12 về 8 dấu cách) ***
        print(f"Error during prediction: {{e}}")
        error = {{"error": str(e)}}
        return jsonify(error), 500 if is_api_call else render_template('index.html', results=error)

def run_server():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
"""

# Ghi nội dung đã được chèn biến vào file
with open("main_app.py", "w", encoding="utf-8") as f:
    f.write(file_content)

print("Đã ghi file main_app.py (với logic pre-loading ĐÃ SỬA LỖI) thành công.")

Đã ghi file main_app.py (với logic pre-loading ĐÃ SỬA LỖI) thành công.


In [15]:
# === Cell 8: Khởi chạy API và Ngrok ===
# SỬA LỖI: Thêm ngrok.kill() và bỏ time.sleep()

from pyngrok import ngrok, conf
import threading
import time

# Ngắt mọi kết nối Ngrok cũ (nếu có) trước khi bắt đầu
print("Ngắt các kết nối Ngrok cũ (nếu có)...")
ngrok.kill()

print("Đang thiết lập Ngrok Auth Token...")
conf.get_default().auth_token = NGROK_AUTH_TOKEN

# --- QUAN TRỌNG ---
# Khi dòng 'import main_app' này chạy, 
# nó sẽ kích hoạt hàm pre_load_all_models() bên trong file đó.
# Output sẽ hiển thị quá trình tải model ở đây.
print("Đang import main_app (sẽ kích hoạt pre-loading model)...")
import main_app

print("Đang khởi chạy Flask server trong background...")
flask_thread = threading.Thread(target=main_app.run_server, daemon=True)
flask_thread.start()

# Không cần time.sleep(3) nữa, vì chúng ta đã chờ model tải xong.

# 2. Khởi tạo Ngrok Tunnel
try:
    print("Đang kết nối Ngrok...")
    public_url = ngrok.connect(5000) # Kết nối tới port 5000
    print("="*60)
    print(f"✅ ỨNG DỤNG WEB CỦA BẠN ĐÃ ONLINE! ✅")
    print(f"Truy cập URL này trên trình duyệt: {public_url}")
    print("="*60)
    print("Tất cả model đã được tải, API sẵn sàng nhận request ngay lập tức.")
except Exception as e:
    print(f"Lỗi khi khởi tạo Ngrok: {e}")
    print("Vui lòng kiểm tra lại NGROK_AUTH_TOKEN ở Cell 2.")

Ngắt các kết nối Ngrok cũ (nếu có)...
Đang thiết lập Ngrok Auth Token...
Đang import main_app (sẽ kích hoạt pre-loading model)...
Flask App running on device: cuda
BẮT ĐẦU TẢI TRƯỚC MODEL SUMMARIZER (WARM-UP)...
Loading TEXT-GENERATION pipeline for: Qwen/Qwen2.5-1.5B-Instruct...


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
INFO:werkzeug:127.0.0.1 - - [14/Nov/2025 14:47:26] "POST /predict HTTP/1.1" 200 -


Using cached model for: restaurant
Using cached generator (LLM).


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
INFO:werkzeug:127.0.0.1 - - [14/Nov/2025 14:47:37] "POST /predict HTTP/1.1" 200 -


Cache miss. Loading model for: hotel...
Model for 'hotel' loaded in 28.92 seconds.
Using cached generator (LLM).


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
INFO:werkzeug:127.0.0.1 - - [14/Nov/2025 14:49:27] "POST /predict HTTP/1.1" 200 -


## Don't Run Last Cell if you just start

In [ ]:
# === Cell 9: Reset Server & Xóa Cache GPU ===
# CHẠY CELL NÀY TRƯỚC KHI CHẠY LẠI CELL 8

import sys
import torch
from pyngrok import ngrok

print("Ngắt kết nối Ngrok cũ...")
ngrok.kill()

# 1. Xóa cache của Python (quan trọng nhất)
# Buộc Python phải import lại file .py khi chạy Cell 8
try:
    if 'main_app' in sys.modules:
        del sys.modules['main_app']
        print("-> Đã xóa module 'main_app' khỏi cache.")
        
    if 'app_logic' in sys.modules:
        del sys.modules['app_logic']
        print("-> Đã xóa module 'app_logic' khỏi cache.")
except Exception as e:
    print(f"Lỗi khi xóa module cache: {e}")

# 2. Giải phóng VRAM của GPU
print("Đang giải phóng VRAM (GPU)...")
torch.cuda.empty_cache()

print("\n✅ Reset hoàn tất!")
print("Bây giờ bạn có thể CHẠY LẠI CELL 8 để tải model mới.")